In [0]:
import asyncio
import json
from pathlib import Path
from typing import Dict, List

import aiohttp
from tenacity import retry, stop_after_attempt, wait_fixed
from tqdm.auto import tqdm

# dataset.jsonを読み込む
with open("dataset.json", "r") as f:
    data = json.load(f)

# PDFの保存先ディレクトリを作成
pdf_dir = Path("../dataset/pdfs")
pdf_dir.mkdir(parents=True, exist_ok=True)


# リトライ設定付きのダウンロード関数
@retry(stop=stop_after_attempt(3), wait=wait_fixed(1))
async def download_pdf(
    session: aiohttp.ClientSession, url: str, save_path: Path
) -> Dict:
    """
    PDFをダウンロードする関数（最大3回リトライ、1秒間隔）
    """
    try:
        async with session.get(url) as response:
            response.raise_for_status()
            content = await response.read()
            save_path.write_bytes(content)
            return {"url": url, "status": "success", "path": str(save_path)}
    except Exception as e:
        return {"url": url, "status": "error", "error": str(e)}


async def download_pdfs_in_batch(
    items: List[Dict], batch_size: int = 5, delay: float = 1.0
):
    """
    PDFを並列バッチでダウンロードする関数
    """
    results = []

    async with aiohttp.ClientSession() as session:
        # 全体の進捗バーを作成
        with tqdm(total=len(items), desc="PDFダウンロード", unit="file") as pbar:
            # バッチごとに処理
            for i in range(0, len(items), batch_size):
                batch = items[i : i + batch_size]

                # バッチ内のタスクを作成
                tasks = []
                for item in batch:
                    # PDFファイル名を生成（URLから抽出）
                    pdf_filename = item["pdf_url"].split("/")[-1]
                    save_path = pdf_dir / pdf_filename

                    # すでにダウンロード済みの場合はスキップ
                    if save_path.exists():
                        results.append(
                            {
                                "url": item["pdf_url"],
                                "status": "skipped",
                                "path": str(save_path),
                            }
                        )
                        pbar.update(1)
                        continue

                    tasks.append(download_pdf(session, item["pdf_url"], save_path))

                # バッチ内のダウンロードを並列実行
                if tasks:
                    batch_results = await asyncio.gather(*tasks, return_exceptions=True)

                    # 結果を処理
                    for result in batch_results:
                        if isinstance(result, Exception):
                            results.append({"status": "error", "error": str(result)})
                        else:
                            results.append(result)
                        pbar.update(1)

                # 次のバッチまで待機（最後のバッチ以外）
                if i + batch_size < len(items):
                    await asyncio.sleep(delay)

    return results


# ダウンロードを実行
dataset_items = data["dataset"]
results = await download_pdfs_in_batch(dataset_items, batch_size=5, delay=1.0)

# 結果のサマリーを表示
success_count = sum(1 for r in results if r["status"] == "success")
skipped_count = sum(1 for r in results if r["status"] == "skipped")
error_count = sum(1 for r in results if r["status"] == "error")

print("\n完了!")
print(f"成功: {success_count}, スキップ: {skipped_count}, 失敗: {error_count}")
print(f"PDFの保存先: {pdf_dir.absolute()}")